In [ ]:
import pandas as pd
import pickle

In [ ]:
Target_list_PPI = pd.read_csv('../preprocessed_data/Target_list_PPI.csv', index_col=None)['Proteins'].unique().tolist()
Target_list_HTMDT = pd.read_csv('../preprocessed_data/Target_list_HTMDT.csv', index_col=None)['Proteins'].unique().tolist()
Target_list = pd.read_csv('../preprocessed_data/Target_list.csv', index_col=None)['Proteins'].unique().tolist()

In [ ]:
Target_list_selected = sorted(list(set(Target_list_PPI) & set(Target_list_HTMDT) & set(Target_list)))
len(Target_list_selected)

In [ ]:
Target_dict = {key: value for value, key in enumerate(Target_list_selected)}

with open('../preprocessed_data/Target_dict.pkl', 'wb') as file:
    pickle.dump(Target_dict, file)

In [ ]:
Target_list_selected = pd.DataFrame(Target_list_selected, columns=['Proteins'])
Target_list_selected.to_csv('../preprocessed_data/Target_list_selected.csv', index=None)

In [ ]:
herb_target = pd.read_csv('../preprocessed_data/Herb-Target.csv', index_col=None)
print(len(herb_target))
herb_target = herb_target[herb_target['Gene symbol'].isin(Target_list_selected['Proteins'].tolist())]
print(len(herb_target))
herb_target.to_csv('../preprocessed_data/Preprocessed_Herb-Target.csv', index=None)

In [ ]:
target_target = pd.read_csv('../preprocessed_data/Target-Target.csv', index_col=None)
print(len(target_target))
target_target = target_target[target_target['node1'].isin(Target_list_selected['Proteins'].tolist())]
target_target = target_target[target_target['node2'].isin(Target_list_selected['Proteins'].tolist())]
print(len(target_target))
target_target.to_csv('../preprocessed_data/Preprocessed_Target-Target.csv', index=None)

In [ ]:
target_target['node1'] = target_target['node1'].map(Target_dict)
target_target['node2'] = target_target['node2'].map(Target_dict)

interaction_to_idx = {interaction: i+1 for i, interaction in enumerate(sorted(target_target['interaction'].unique().tolist()))}
with open('../preprocessed_data/interaction_to_idx.pkl', 'wb') as file:
    pickle.dump(interaction_to_idx, file)

target_target['interaction'] = target_target['interaction'].map(interaction_to_idx)

num_nodes = len(Target_dict)
self_loops = pd.DataFrame({
    'node1': range(num_nodes),
    'node2': range(num_nodes),
    'interaction': [0] * num_nodes
})

ppis_expanded = pd.concat([target_target, self_loops], ignore_index=True)

ppis_expanded.to_csv('../preprocessed_data/kg_v1.csv', index=None)

In [ ]:
interaction_to_idx

In [ ]:
HTMDT = pd.read_csv('../preprocessed_data/HTMDT.csv', index_col=None)
print(len(HTMDT))
HTMDT = HTMDT[(HTMDT['node2_type'] != 'Target') | ((HTMDT['node2_type'] == 'Target') & (HTMDT['node2'].isin(Target_list_selected['Proteins'].tolist())))]
print(len(HTMDT))
HTMDT.to_csv('../preprocessed_data/Preprocessed_HTMDT.csv', index=None)